In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import sys
import datetime as dt
import re
from datetime import datetime, timedelta


In [12]:
import pickle
def save(obj, name, path ):
    with open(path+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load(name ,path):
    with open(path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [15]:
save(teams_dict,'teams_dict','C:/Users/clement.laplace/Documents/Perso/EPL Pred/Data/Dictionnaries/')

test=load('teams_dict','C:/Users/clement.laplace/Documents/Perso/EPL Pred/Data/Dictionnaries/')

# Data

## Dictionnaries

In [2]:
teams_dict={'ARS':'arsenal',    
       'BOU': 'bournemouth',
       'BHA':'brighton & hove albion',
       'BUR':'burnley',
       'CHE': 'chelsea',
       'CRY': 'crystal palace',
       'EVE': 'everton',
       'HUD': 'huddersfield town',
       'LEI': 'leicester city',
       'LIV': 'liverpool',
       'MCI': 'manchester city',
       'MUN': 'manchester united',
       'NEW': 'newcastle united',
       'Norwich': 'norwich',
       'Sheffield': 'sheffield United',
       'SOU': 'southampton',
       'STK': 'stoke city',
       'SWA':'swansea city',
       'TOT': 'tottenham hotspur',
       'Villa': 'aston villa',
       'WAT': 'watford',
       'WBA': 'west bromwich albion',
       'WHU': 'west ham united',
       'Wolverhampton': 'wolverhampton'
      }


In [64]:
tweet_dict={'ARS':['@arsenal','@Arsenal', '#AFC','#afc','#Arsenal','#COYG','#coyg','#Gunners','#WeAreTheArsenal','#Gooners','#thegunners','#comeonarsenal'],
       
       'BOU': ['@afcbournemouth','#afcb', '#bournemouth','#thecherries'],
       'BHA':['@OfficialBHAFC','#BrightonAndHoveAlbionFC','#BHAFC','#bhafc'],
       'BUR':['@BurnleyOfficial','#twitterclarets','#BurnleyFC','#Clarets'],
       'CHE': ['@ChelseaFC','#cfc','#CFC', '#chelsea','#ChelseaFC','#Chelsea'],
       'CRY': ['@CPFC','#CPFC','#cpfc', '#crystalpalace','#CrystalPalace'],
       'EVE': ['@Everton','#efc','#EFC','#coyb', '#everton','#Everton','#EvertonFC','#evertonfc','#COYB'],
       'HUD': ['#htafc','@htafc','@HTAFC','@htafcdotcom','#HTAFC'],
       'LEI': ['@LCFC','#lcfc','#LCFC', '#leicesterfc', '#leicestercity',
                    '#leicestercityfc'],
       'LIV': ['@LFC','#LFC','#Lfc','#lfc', '#YNWA','#youwillneverwalkalone','#liverpoolfc','#LiverpoolFC', '#livfc'],
       'MCI': ['@ManCity','@Mancity','#ManCity','#Mancity','#MANCHESTERCITY','#ManchesterCity','#mancityfc','#MCFC','#mcfc', '#manchestercity', '#mancity','#TheSkyBlues','#manchestercityfc'],
       'MUN': ['@ManUtd','#MUFC','#mufc', '#manchesterunited', '#manutd','#GGMU','#manunited','#ManUnited','#GGMU','#ManchesterUnited'],
       'NEW': ['@NUFC','#nufc','#NUFC','#ToonArmy', '#newcastlefc', '#newcastleunited',
                    '#newcastleunitedfc','#TheMagpies'],
       'Norwich': ['@NorwichCityFC','#ncfc', '#norwichcity'],
       'Sheffield': ['@SheffieldUnited','#sufc', '#twitterblades'],
       'SOU': ['@SouthamptonFC','#saintsfc','#SaintsFC', '#southamptonfc'],
       'STK': ['@stokecity','#SCFC','#scfc','#stokecity'],
       'SWA':['@swansofficial','SwansOfficial','#swans','#Swans','#jackarmy'],
       'TOT': ['@SpursOfficial','@spursofficial','@SpursOfficial.','#COYS','#coys','#THFC','#Tottenham','#TottenhamHotspur', '#tottenhamfc', '#spursfc',
                    '#hotspurfc', '#tottenhamhotspurfc','#thfc','#THFC','#Coys'],
       'Villa': ['@AVFCOfficial','#avfc', '#astonvilla', '#astonvillafc', '#villafc','#AVFC'],
       'WAT': ['@WatfordFC','#WatfordFC','#TheHornets','#watfordfc'],
       'WBA': ['@WBA','#WBA','#WBAFC','#wbafc'],
       'WHU': ['@WestHamUtd','#COYI','#coyi','#WestHam','#whufc', '#westhamfc', '#westhamunited', '#westhamunitedfc',
                  '#whu','#WHUFC','#whufc'],
       'Wolverhampton': ['@Wolves','#WolverhamptonWanderers','#wwfc']
      }


## Fixtures

In [4]:
fixtures= pd.read_excel(r'C:\Users\clement.laplace\Documents\Perso\EPL Pred\Data\fixtures 1718.xlsx')

In [5]:
Htri=[(list(teams_dict.keys())[list(teams_dict.values()).index(teams.lower())])for teams in fixtures['HOME TEAM']]  # Prints george
Atri=[(list(teams_dict.keys())[list(teams_dict.values()).index(teams.lower())])for teams in fixtures['AWAY TEAM']]
fixtures['HTRI']=Htri
fixtures['ATRI']=Atri
fixtures['GAME WEEK']=(fixtures.index/10+1).astype(int)
for i in range(len(fixtures)):
    fixtures['FULL_DATE']=datetime.combine(fixtures.DATE.iloc[i],fixtures.TIME.iloc[i])

In [6]:
fixtures_train=fixtures[fixtures['GAME WEEK']>=36]
fixtures_train=fixtures_train.reset_index(drop=True)

In [7]:
fixtures_train.head()

,DIVISION,DATE,TIME,HOME TEAM,AWAY TEAM,HTRI,ATRI,GAME WEEK,FULL_DATE
0,EPL,2018-04-28,15:00:00,Burnley,Brighton & Hove Albion,BUR,BHA,36,2018-05-13 15:00:00
1,EPL,2018-04-28,15:00:00,Crystal Palace,Leicester City,CRY,LEI,36,2018-05-13 15:00:00
2,EPL,2018-04-28,15:00:00,Huddersfield Town,Everton,HUD,EVE,36,2018-05-13 15:00:00
3,EPL,2018-04-28,15:00:00,Liverpool,Stoke City,LIV,STK,36,2018-05-13 15:00:00
4,EPL,2018-04-28,15:00:00,Manchester United,Arsenal,MUN,ARS,36,2018-05-13 15:00:00


## Tweets

In [8]:
# Tweets & Sentiments from BetSentiment
tweets = pd.read_csv(r'C:\Users\clement.laplace\Documents\Perso\EPL Pred\Data\EPL-Team-sentiment\betsentiment-EN-tweets-sentiment-teams-split\Teams Tweet.csv', encoding='latin1')
# Get tweets before end of the season 2017-2018
tweets_18 = tweets[tweets['tweet_date_created']<'2018-06-01']
tweets_18=tweets_18.sort_values(by='tweet_date_created')

In [66]:
#Get all hashtags/mentions & Attribute them to teams for classification
tweets_18['Team Mentions']=[re.findall(r'[#@][^\s#@,.?!;]+', i) for i in tweets_18.tweet_text]
tweets_18['team']=[list(set(k for k,v in tweet_dict.items() for i in mentions if i in v)) for mentions in tweets_18['Team Mentions']]
#Reformat Date to datetime value for processing when merging with Fixtures
tweets_18['tweet_date_created']=pd.to_datetime(tweets_18['tweet_date_created'])
tweets_18=tweets_18.reset_index(drop=True)

In [11]:
#Duplication of tweets if different teams hashtags in tweet
df_tweets =pd.DataFrame([(row['tweet_date_created'], d) for _, row in tqdm(tweets_18.iterrows()) 
                           for d in row['team']],
                          columns=['tweet_date_created', 'team'])
df_text =pd.DataFrame([(row['tweet_text'], d) for _, row in tqdm(tweets_18.iterrows())
                           for d in row['team']],
                          columns=['tweet_text', 'team'])
df_rating= pd.DataFrame([(row['sentiment_score'], d) for _, row in tqdm(tweets_18.iterrows()) 
                           for d in row['team']],
                          columns=['sentiment_score', 'team'])
df_sentiment=pd.DataFrame([(row['sentiment'], d) for _, row in tqdm(tweets_18.iterrows()) 
                           for d in row['team']],
                          columns=['sentiment', 'team'])

In [12]:
# Computation of sentiment variables

In [13]:
def positivity (text):
    try: 
        return float(re.search('"Positive":(.*?),"', text).group(1))
    except:
        return 0        
def mixity (text):
    try: 
        return float(re.search('"Mixed":(.*?)}', text).group(1))
    except:
        return 0        

In [14]:
df_tweets['tweet']=df_text['tweet_text']
df_tweets['rating']=df_rating['sentiment_score']
df_tweets['overall_sentiment']= df_sentiment['sentiment']

In [15]:
neutral=[float(re.search('"Neutral":(.*),"Neg', rating).group(1)) for rating in df_tweets.rating]
negative=[float(re.search('"Negative":(.*?),"', rating).group(1)) for rating in df_tweets.rating]
positive=[ positivity(rating) for rating in df_tweets.rating]
mixed=[mixity(rating) for rating in df_tweets.rating]
df_tweets['positive']=positive
df_tweets['negative']=negative
df_tweets['neutral']=neutral
df_tweets['mixed']=mixed
df_tweets=pd.get_dummies(df_tweets, columns=['overall_sentiment'],prefix='',prefix_sep='')

## FIFA Teams Ratings

In [16]:
# FIFA ratings
fifa= pd.read_excel(r'C:\Users\clement.laplace\Documents\Perso\EPL Pred\Data\FIFA team.xlsx')

In [17]:
fifa.sort_values(by='Nom')

,Nom,Championnat,ATT,MIL,DÉF,GEN
2,Arsenal,Barclays PL,86,82,80,83
15,Bournemouth,Barclays PL,76,74,75,75
18,Brighton,Barclays PL,74,74,73,74
16,Burnley,Barclays PL,74,75,75,75
0,Chelsea,Barclays PL,85,85,82,84
12,Crystal Palace,Barclays PL,81,76,75,76
6,Everton,Barclays PL,79,82,79,80
19,Huddersfield,Barclays PL,73,73,72,73
8,Leicester City,Barclays PL,79,79,76,78
5,Liverpool,Barclays PL,83,82,80,82


## EPL Season Data

In [18]:
# Fixtures data season
seasons_data=pd.read_csv(r'C:\Users\clement.laplace\Documents\Perso\EPL Pred\Data\EPLDataset.csv')
seasons_data['Final_result']=0
for i in range(len(seasons_data)):
    if seasons_data.loc[i,['home_goals']][0]>seasons_data.loc[i,['away_goals']][0]:
        seasons_data.loc[i,['Final_result']]=1
    if seasons_data.loc[i,['home_goals']][0]<seasons_data.loc[i,['away_goals']][0]:
        seasons_data.loc[i,['Final_result']]=-1
season_18=seasons_data[seasons_data['season']=='2017/2018']

In [19]:
season_18['game_week']=season_18.matchweek.str.extract('(\d+)')
season_18['game_week']=pd.to_numeric(season_18['game_week'],errors='ignore')
season_18_hist=season_18[season_18['game_week']<36]
season_18_2= season_18[season_18['game_week']>=36]
season_18_2=season_18_2.reset_index(drop=True)

C:\Users\clement.laplace\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\clement.laplace\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
season_18_home=season_18_hist.groupby(by='home_team').mean().reset_index()
season_18_away=season_18_hist.groupby(by='away_team').mean().reset_index()

## FINAL DATASET

In [21]:
columns=['DIVISION', 'DATE', 'TIME', 'HOME TEAM', 'AWAY TEAM', 'HTRI', 'ATRI',
       'GAME WEEK', 'FULL_DATE', 'POSITIVE_home', 'NEGATIVE_home',
       'NEUTRAL_home', 'MIXED_home', 'positive_sum_home', 'negative_sum_home',
       'neutral_sum_home', 'mixed_sum_home', 'positive_mean_home',
       'negative_mean_home', 'neutral_mean_home', 'mixed_mean_home',
       'TOTAL_home', 'POSITIVE_away', 'NEGATIVE_away', 'NEUTRAL_away',
       'MIXED_away', 'positive_sum_away', 'negative_sum_away',
       'neutral_sum_away', 'mixed_sum_away', 'positive_mean_away',
       'negative_mean_away', 'neutral_mean_away', 'mixed_mean_away',
       'TOTAL_away']


In [22]:
def sentiment_dataset(tweets, team, date):
    mask = (tweets['team']==team)&(tweets['tweet_date_created']>date-timedelta(3))&(tweets['tweet_date_created']<=date)
    sentiment_sum = tweets[mask].groupby('team').agg({'positive':'sum', 'negative':'sum','neutral':'sum','mixed':'sum'}) \
           .rename(columns={'positive':'positive_sum','negative':'negative_sum','neutral':'neutral_sum','mixed':'mixed_sum'}) \
           .reset_index()
    sentiment_mean = tweets[mask].groupby('team').agg({'positive':'mean', 'negative':'mean','neutral':'mean','mixed':'mean'}) \
           .rename(columns={'positive':'positive_mean','negative':'negative_mean','neutral':'neutral_mean','mixed':'mixed_mean'}) \
           .reset_index()
    sentiment_count = tweets[mask].groupby('team').agg({'POSITIVE':'sum', 'NEGATIVE':'sum','NEUTRAL':'sum','MIXED':'sum'}) \
           .reset_index()
    sentiment= sentiment_count.merge(sentiment_sum, on='team',how='left')
    sentiment= sentiment.merge(sentiment_mean, on='team',how='left')
    sentiment['TOTAL']= sentiment['POSITIVE']+sentiment['NEGATIVE']+sentiment['NEUTRAL']+sentiment['MIXED']
    sentiment=sentiment.drop(columns=['team'])
    
    return sentiment.loc[0].values.tolist()
def nlp_dataset_creation_GW (new_df_columns,tweets,fixtures):
    df_final=pd.DataFrame(columns=columns)
    for i in range(len(fixtures_train)):
        df_final.loc[i,['DIVISION', 'DATE', 'TIME', 'HOME TEAM', 'AWAY TEAM', 'HTRI', 'ATRI',
                        'GAME WEEK', 'FULL_DATE']]=fixtures_train.iloc[i]
        df_final.loc[i,['POSITIVE_home', 'NEGATIVE_home',
                        'NEUTRAL_home', 'MIXED_home', 'positive_sum_home', 'negative_sum_home',
                        'neutral_sum_home', 'mixed_sum_home', 'positive_mean_home',
                        'negative_mean_home', 'neutral_mean_home', 'mixed_mean_home',
                        'TOTAL_home']]=sentiment_dataset(df_tweets,fixtures_train.loc[i,['HTRI']][0],
                                                         fixtures_train.loc[i,['FULL_DATE']][0])
        df_final.loc[i,['POSITIVE_away', 'NEGATIVE_away', 'NEUTRAL_away',
                        'MIXED_away', 'positive_sum_away', 'negative_sum_away',
                        'neutral_sum_away', 'mixed_sum_away', 'positive_mean_away',
                        'negative_mean_away', 'neutral_mean_away', 'mixed_mean_away',
                        'TOTAL_away']]=sentiment_dataset(df_tweets,fixtures_train.loc[i,['ATRI']][0],
                                                         fixtures_train.loc[i,['FULL_DATE']][0])
    return df_final

In [23]:
df_final=nlp_dataset_creation_GW(columns,df_tweets,fixtures_train)

In [24]:
df_final=df_final.merge(season_18_2[['home_team','game_week','Final_result']],left_on=['GAME WEEK','HTRI'], right_on=['game_week','home_team'], how='left').drop(columns=['game_week','home_team'])

In [25]:
df_final.head()

,DIVISION,DATE,TIME,HOME TEAM,AWAY TEAM,HTRI,ATRI,GAME WEEK,FULL_DATE,POSITIVE_home,...,positive_sum_away,negative_sum_away,neutral_sum_away,mixed_sum_away,positive_mean_away,negative_mean_away,neutral_mean_away,mixed_mean_away,TOTAL_away,Final_result
0,EPL,2018-04-28 00:00:00,15:00:00,Burnley,Brighton & Hove Albion,BUR,BHA,36,2018-05-13 15:00:00,97,...,115.45,28.64,200.303,9.60774,0.326129,0.080904,0.565826,0.0271405,98,0
1,EPL,2018-04-28 00:00:00,15:00:00,Crystal Palace,Leicester City,CRY,LEI,36,2018-05-13 15:00:00,213,...,84.9024,44.3193,185.186,11.5924,0.260437,0.135949,0.568055,0.0355596,70,1
2,EPL,2018-04-28 00:00:00,15:00:00,Huddersfield Town,Everton,HUD,EVE,36,2018-05-13 15:00:00,370,...,354.544,239.76,913.957,63.7382,0.225537,0.152519,0.581398,0.0405459,1572,-1
3,EPL,2018-04-28 00:00:00,15:00:00,Liverpool,Stoke City,LIV,STK,36,2018-05-13 15:00:00,1197,...,93.9517,68.2826,189.795,19.9705,0.252558,0.183555,0.510202,0.053684,116,0
4,EPL,2018-04-28 00:00:00,15:00:00,Manchester United,Arsenal,MUN,ARS,36,2018-05-13 15:00:00,1595,...,869.286,416.066,1631.9,124.746,0.285761,0.136774,0.536457,0.041008,3042,1


In [26]:
df_final.to_csv('df_final.csv')

# DATA EXPLORATION

In [27]:
comparaison=pd.read_excel('df_final.xlsx',sheet_name='Comparaison')

In [28]:
import seaborn as sns
corr = comparaison.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

C:\Users\clement.laplace\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [29]:
comparaison.corr()

,POSITIVE,NEGATIVE,NEUTRAL,MIXED,positive_sum,negative_sum,neutral_sum,mixed_sum,positive_mean,negative_mean,neutral_mean,mixed_mean,TOTAL,Final_result
POSITIVE,1.000000,0.708371,0.800018,0.775212,1.000000,0.665184,0.864253,0.796391,-0.221719,0.260680,-0.049774,0.185520,0.796391,0.368204
NEGATIVE,0.708371,1.000000,0.847066,0.791360,0.708371,0.983690,0.845919,0.918262,-0.529559,0.573819,0.116916,0.502049,0.779132,0.270552
NEUTRAL,0.800018,0.847066,1.000000,0.919407,0.800018,0.866071,0.934853,0.872872,-0.413493,0.464286,0.143823,0.395515,0.872872,0.455346
MIXED,0.775212,0.791360,0.919407,1.000000,0.775212,0.781496,0.914056,0.842650,-0.335539,0.505674,0.057852,0.428102,0.983092,0.405287
positive_sum,1.000000,0.708371,0.800018,0.775212,1.000000,0.665184,0.864253,0.796391,-0.221719,0.260680,-0.049774,0.185520,0.796391,0.368204
negative_sum,0.665184,0.983690,0.866071,0.781496,0.665184,1.000000,0.800018,0.872872,-0.548327,0.598214,0.143823,0.530349,0.736485,0.310023
neutral_sum,0.864253,0.845919,0.934853,0.914056,0.864253,0.800018,1.000000,0.933700,-0.357466,0.395515,0.085973,0.321267,0.933700,0.368204
mixed_sum,0.796391,0.918262,0.872872,0.842650,0.796391,0.872872,0.933700,1.000000,-0.439388,0.463713,0.027462,0.384465,0.861111,0.281181
positive_mean,-0.221719,-0.529559,-0.413493,-0.335539,-0.221719,-0.548327,-0.357466,-0.439388,1.000000,-0.548327,-0.592760,-0.493213,-0.302079,-0.363327
negative_mean,0.260680,0.573819,0.464286,0.505674,0.260680,0.598214,0.395515,0.463713,-0.548327,1.000000,0.143823,0.800018,0.463713,0.164700
